### Imports

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import os
from pathlib import Path
import librosa
import torch
from transformers import (
    AutoModelForSpeechSeq2Seq,
    AutoProcessor,
    pipeline,
)
import pandas as pd

### Lecture des metadata

In [14]:
# Répertoire du script
path = Path("/content/drive/MyDrive/911")
os.chdir(path)

# Charger les données
df_meta = pd.read_csv("./911_metadata.csv")
# Extraire les appels qui n'ont pas nan dans false alarm, et qui possède un référence fichier
df_meta = df_meta.dropna(subset=["false_alarm", "file_name"])
df_meta.shape
df_meta.head()

,id,link,title,date,state,civilian_initiated,deaths,potential_death,false_alarm,description,file_name
0,1,https://web.archive.org/web/20150417085342/htt...,North Hollywood bank robbery,2/97,California,0.0,2.0,1.0,0.0,– The unforgettable collection of radio loggin...,call_1.wav
1,2,https://web.archive.org/web/20150417085342/htt...,Detroit Child’s 911 Call – audio,2/06,Michigan,1.0,1.0,1.0,0.0,– In Feb. 2006 5 year-old Robert Turner called...,call_2.wav
7,8,https://web.archive.org/web/20150417085342/htt...,Girl’s Murder 911 Call,3/05,Florida,1.0,2.0,1.0,0.0,– the 911 call of a lifetime. Volusia County (...,call_8.wav
8,9,https://web.archive.org/web/20150417085342/htt...,‘Shoot Her?’ 911 call,4/05,Texas,1.0,0.0,0.0,1.0,– caller reports her daughter is creating a di...,call_9.wav
9,10,https://web.archive.org/web/20150417085342/htt...,Snowstorm 911 Call,1/05,Nebraska,1.0,2.0,1.0,0.0,– a couple under the influence of drugs dialed...,call_10.wav


### Ajouter la taille du fichier dans le df

In [15]:
# Ajouter la taille du fichier dans le df
df_meta["file_size"] = df_meta["file_name"].apply(
    lambda x: Path(path.joinpath(x)).stat().st_size
)

In [16]:
# proportion de false alarm
df_meta["false_alarm"].value_counts(normalize=True, dropna=False)

,proportion
false_alarm,
0.0,0.948718
1.0,0.051282


In [17]:
# Trier le df par ordre croissant de file_size
df_meta = df_meta.sort_values("file_size", ascending=True)
df_meta.head()

,id,link,title,date,state,civilian_initiated,deaths,potential_death,false_alarm,description,file_name,file_size
58,59,https://web.archive.org/web/20150417085342/htt...,#1,12/07,Nebraska,1.0,9.0,1.0,0.0,– In Dec. 2007 Robert Hawkins entered the West...,call_59.wav,316054
280,282,https://web.archive.org/web/20150417085349/htt...,Murder-suicide,4/13,Michigan,1.0,3.0,1.0,0.0,– In April 2013 a woman in Newaygo County (Mic...,call_282.wav,403826
297,299,https://web.archive.org/web/20150417085349/htt...,Shooting,6/13,California,1.0,0.0,1.0,0.0,– In June 2013 a Bakersfield (Calif.) man dial...,call_299.wav,734850
399,401,https://web.archive.org/web/20150417085338/htt...,Dog dials 911,9/08,Arizona,1.0,0.0,1.0,0.0,"– In Sept. 2008 “Buddy,” a trained German Shep...",call_401.wav,998722
736,740,https://web.archive.org/web/20150417085349/htt...,Call from murder suspect,8/10,California,1.0,0.0,1.0,0.0,– In August 2010 the suspect in the murder of ...,call_740.wav,1065678


In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# wer_metric = evaluate.load("wer")

model_ids = [
    "openai/whisper-tiny",
    "openai/whisper-base",
    "openai/whisper-small",
    "openai/whisper-medium",
    "openai/whisper-large",
    "openai/whisper-large-v2",
    "openai/whisper-large-v3",
    "openai/whisper-large-v3-turbo",
]

In [ ]:
modele = "openai/whisper-small"

for index, row in df_meta.iterrows():
    audio_file = row["file_name"]
    audio_title = row["title"]
    audio_id = row["id"]

    urgent = "urgent" if not row["false_alarm"] else "non_urgent"

    audio_path = path.joinpath(f"{audio_file}")

    print(f"\t\tTranscribing file {audio_file}...")

    audio_data_test, sr_test = librosa.load(audio_path, sr=16000)

    tmp_model = AutoModelForSpeechSeq2Seq.from_pretrained(
        modele, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    ).to(device)

    tmp_processor = AutoProcessor.from_pretrained(modele)

    tmp_pipe = pipeline(
        task="automatic-speech-recognition",
        model=tmp_model,
        tokenizer=tmp_processor.tokenizer,
        feature_extractor=tmp_processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
        return_timestamps=True,
    )

    output = tmp_pipe(audio_data_test)

    # Écrire le fichier texte correspondant

    with open(
        f"./trs/{audio_file}-{urgent}.txt", "w"
    ) as f:
        f.write(output["text"])

		Transcribing file call_59.wav...


Device set to use cuda
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


		Transcribing file call_282.wav...


Device set to use cuda


		Transcribing file call_299.wav...


...

Device set to use cuda


		Transcribing file call_627.wav...


Device set to use cuda


		Transcribing file call_147.wav...


Device set to use cuda
